# BMI 203: Step 3: Make Your Classifier

This Jupyter notebook classifies Rap1 transcription factors using a neural network.

### Import modules.

In [1]:
from nn import nn, preprocess, io
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
%load_ext autoreload
%autoreload 2

Initalize seed for consistent results.

In [3]:
seed = 15

### Use read functions from io.py to read in positive and negative motif examples.

In [4]:
pos = io.read_text_file("data/rap1-lieb-positives.txt")
neg = io.read_fasta_file("data/yeast-upstream-1k-negative.fa")

In [5]:
len(pos)

137

Determine lengths of positive and negative sequences.

In [6]:
for seq in range(5):
    print( len(pos[seq]) )

17
17
17
17
17


In [7]:
for seq in range(5):
    print( len(neg[seq]) )

1000
1000
1000
1000
1000


#### Make lengths of negative sequences equal those of positive sequences.

In [8]:
new_neg = []
for seq in neg:
    len_17 = seq[:17]
    new_neg += [len_17]

In [9]:
for seq in range(5):
    print( len(new_neg[seq]) )

17
17
17
17
17


Combine positve and shortened negative sequences into one list and generate corresponding labels.

In [10]:
seqs = list(pos) + list(new_neg)
len(seqs)

3300

In [13]:
labels = [True] * len(pos) + [False] * len(new_neg)
len(labels)

3300

### Balance classes using sample_seq function.

My sampling method with replacement defines two arrays: one of the positive and one of the negative sequence indices of the concatenated list of sequences. I then perform a "coin-flip", and randomly generate a float value from 0 to 1.0 from a uniform distribution, then evaluate if that value is above or below 0.5. For each sequence in the concatenated list of sequences, sample a posititive sequence if value is below 0.5, otherwise sample a negative sequence. Ultimately, this should yield a sampling of sequences with relatively balanced classes.

In [21]:
sampled_seqs, sampled_labels = preprocess.sample_seqs(seqs, labels)

In [22]:
len(sampled_seqs)

3300

In [23]:
len(sampled_labels)

3300

Check to see if classes are relatively balanced.

In [28]:
pos_labs = []
neg_labs = []
for lab in sampled_labels:
    if lab == True:
        pos_labs += [lab]
    else:
        neg_labs += [lab]

In [30]:
len(pos_labs)

1631

In [31]:
len(neg_labs)

1669

### One-hot encode sampled labels.

In [44]:
one_hot_sampled = preprocess.one_hot_encode_seqs(sampled_seqs)

In [45]:
one_hot_sampled[1:5]

array([[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0]])

In [46]:
len( one_hot_sampled[0] ) == 17 * 4

True

### Split data into training and validation sets.

Split the data into 75% training, and 25% testing sets.

In [48]:
X_train, X_test, y_train, y_test = train_test_split(one_hot_sampled, one_hot_sampled, test_size = 0.25, random_state = 0)

In [49]:
np.shape(X_train)

(2475, 68)

In [50]:
np.shape(X_test)

(825, 68)

In [51]:
X_train[:3]

array([[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0]])

In [52]:
y_train[:3]

array([[0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0]])

### Determine best hyperparameters.

I generate my classifier using sigmoid as the activation function and BCE as the loss function since the output is binary and the task is akin to logistic regression. Here I test a series of hyperparameters in my classifer to determined the optimal learning rate, batch size, and number of epochs. The architecture of my classifer has 3 layers total (input layer, two hidden layers, and binary output layer). This means my neural network should be 68x34x17x1.

In [90]:
# Initialize nn architecture.
nn_arch = [{'input_dim': 68, 'output_dim': 34, 'activation': 'sigmoid'},
           {'input_dim': 34, 'output_dim': 17, 'activation': 'sigmoid'},
           {'input_dim': 17, 'output_dim': 1, 'activation': 'sigmoid'}
          ]

In [ ]:
# Create series of hyperparameters to test. Testing less due to computation time requirement.
lrs = [0.001, 0.0001, 0.00001]
batch_sizes = [10, 25, 50]
epochs = [100, 200, 500]

# Initialize list losses and parameters test runs.
losses = []
test_runs = []

# Test hyperparameters in series.
for lr in lrs:
    for bs in batch_sizes:
        for e in epochs:
            # Generate autoenconder with current hyperparameters.
            test_classifer = nn.NeuralNetwork(nn_arch,
                                              lr = lr,
                                              batch_size = bs,
                                              epochs = e,
                                              seed = seed,
                                              loss_function = "bce")
            loss_train, loss_val = test_classifer.fit(X_train, y_train, X_test, y_test) # Train classifier.
            min_loss = min(loss_val) # Add minimal loss value to store best final model.
            test_runs.append([lr, bs, e, min_loss]) # Append list of hyperparameters and loss to full list.

In [ ]:
test_runs

In [ ]:
# Convert list of potential optimal hyperparameters to dataframe for easy viewing.
optimal_df = pd.DataFrame(test_runs)
optimal_df.columns = ['Learning Rate', 'Batch Size', 'Epochs', 'Validation BCE']
optimal_df.sort_values('Validation MSE').head(3)

#### The best combination of hyperparameters that yield the minimum validation mse loss are:
<br>
Learning Rate: 0.0001
<br>
Batch Size: 10
<br>
Epochs: 500

### Plot training and validation loss by epoch for best network.

In [ ]:
# Define function to plot training and validation losses.
def plot_losses(loss_train, loss_val):
    plt.plot(loss_train, label='Training Loss')
    plt.plot(loss_val, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('BCE Loss')
    plt.legend()
    plt.show()

In [91]:
nn_arch

[{'input_dim': 68, 'output_dim': 34, 'activation': 'relu'},
 {'input_dim': 34, 'output_dim': 17, 'activation': 'relu'},
 {'input_dim': 17, 'output_dim': 1, 'activation': 'sigmoid'}]

In [92]:
# Generate nueral network for classifer using best hyperparameters.
classifier = nn.NeuralNetwork(nn_arch, lr = 0.0001, seed = seed, batch_size = 10, epochs = 500, loss_function = "mse")
classifier_loss_train, classifier_loss_val = classifier.fit(X_train, y_train, X_test, y_test)

ValueError: shapes (10,68) and (1,17) not aligned: 68 (dim 1) != 1 (dim 0)

In [ ]:
# Plot losses side by side.
plot_losses(classifier_loss_train, classifier_loss_val)

### Quantify accuracy of classifier.

In [ ]:
prediction = classifier.predict(X_test)
prediction_error = mean_squared_error(y_test, prediction)
print("Average classifier prediction error: ", prediction_error)